In [1]:
# new_ex_14: 한국관광공사의 대한민국 구석구석 크롤링
# https://korean.visitkorea.or.kr/main/main.html

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os   

In [2]:
#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("대한민국 구석구석 사이트의 모든 여행지 정보 수집하기")
print("=" *80)

query_txt = '대한민국구석구석'


cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)

# 소주제 1: 현재 크롤링 시점의 날짜로 폴더 이름을 자동으로 생성하기

f_dir = input("2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

대한민국 구석구석 사이트의 모든 여행지 정보 수집하기
1.크롤링 할 건수는 몇건입니까?: 15
2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\temp\):c:\doit\data\


In [6]:

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

path = "c:/doit/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/main/main.html')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

# 팝업 공지 해제 (코로나)
driver.find_element_by_xpath('''//*[@id="safetyStay1"]/div/div/div/button''').click()

time.sleep(1)

#Step 4. 모든 여행지 링크를 선택합니다

driver.find_element_by_xpath('''//*[@id="btnMenu"]''').click()
driver.find_element_by_link_text('''여행지''').click()
time.sleep(3)

#Step 5 지역 중에서 제주도 버튼 클릭
# driver.find_element_by_xpath("""//*[@id="39"]/a""").click()
try : 
    driver.find_element_by_xpath('''//*[@id="39"]/a''').click()
except :
    driver.find_element_by_xpath('''//*[@id="areaselect"]/option[18]''').click()


In [7]:
#시간을 계산하기 위해 현재 타임스탬프 찍기
now1 = time.localtime()
s1 = '%04d-%02d-%02d-%02d:%02d:%02d' % (now1.tm_year, now1.tm_mon, now1.tm_mday, now1.tm_hour, now1.tm_min, now1.tm_sec)


# 소주제 2: 요청건수와 검색 건수 맞추어서 검색한 후 저장 하기

# Step 6
# 사용자 요청 건수가 실제 검색 건수보다 많을 경우
# 실제 검색 건수로 리셋하기

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result_cnt= soup.find('div', class_='total_check').find('span').get_text( )

search_cnt = int(result_cnt)

if cnt > search_cnt :
    cnt = search_cnt

print("전체 검색 결과 건수 :",search_cnt,"건")
print("실제 최종 출력 건수",cnt)

print("\n")
page_cnt = math.ceil(cnt / 10)
print("크롤링 할 총 페이지 번호: ",page_cnt)
print("=" *80)

전체 검색 결과 건수 : 978 건
실제 최종 출력 건수 15


크롤링 할 총 페이지 번호:  2


In [9]:
# 추출할 게시글의 목록 만들기 10개까지만 되서 완성은 아님

item =[] 
count = 0

item_list = soup.find('ul','list_thumType flnon').find_all('li','bdr_nor')
#print(item_list)

for i in item_list :
    item_list_2 = i.find('div','area_txt').find('div','tit')
    print(item_list_2)

<div class="tit"> <a href="javascript:" onclick='goDetail("95ed115d-ca18-42bc-9540-8c45b6f72941","A01","A0101","39");'>남원 큰엉해안</a> </div>
<div class="tit"> <a href="javascript:" onclick='goDetail("ee7ed31f-0cdc-402d-b415-dbef6fbc8acc","A05","A0502","39");'>성산보말죽칼국수</a> </div>
<div class="tit"> <a href="javascript:" onclick='goDetail("7ecc056b-eed2-4c07-8da2-8fcf882e8037","A02","A0201","39");'>귤림서원</a> </div>
<div class="tit"> <a href="javascript:" onclick='goDetail("6631313a-9c04-44b0-83d9-d0e118e1ed84","A05","A0502","39");'>대원가</a> </div>
<div class="tit"> <a href="javascript:" onclick='goDetail("4b0dd3af-19b4-445f-9623-03b5511905a9","A05","A0502","39");'>24시제주해장국</a> </div>
<div class="tit"> <a href="javascript:" onclick='goDetail("00f44870-acd6-492b-a4fc-aaba95cfda5b","A01","A0101","39");'>형제섬</a> </div>
<div class="tit"> <a href="javascript:" onclick='goDetail("a348a28d-b282-481c-9ebe-9b29d9dd65ee","A03","A0302","39");'>한라산컨트리클럽</a> </div>
<div class="tit"> <a href="javascript:" on

In [20]:
content_list = soup.find('div','wrap_contView')
content_top = content_list.find('div','area_txtView').find('div','inr_wrap').find('div','inr')

print(content_list)
print('='*80)
print(content_top)

<div class="wrap_contView">
<h3>기본정보</h3>
<div class="area_txtView top ms_detail" style="padding-bottom: 54px;">
<div class="inr_wrap" style="overflow: hidden; height: 150px;">
<div class="inr">
<p>''엉''이란 제주도 방언으로 언덕을 뜻하는데, 남원 큰엉은 큰 바위가 바다를 집어 삼킬듯이 입을 크게 벌리고 있는 언덕이라 하여 붙여진 명칭으로 절벽 위는 평지로 부드러운 잔디가 깔려 있다. 바다를 향하여 높이 30m, 길이 200m의 기암절벽이 오랜 세월동안 쉬지 않고 밀려오는 파도를 감싸안은 2개의 자연동굴을 보노라면 세속을 떠난듯한 다른 세계, 때 묻지 않은 자연에 대한 감동을 느낄 수 있다. 해안절벽을 따라 2km의 산책길, 전망대, 간이휴게실, 기초체력단련시설, 화장실 등이 마련되어 있다. 갯바위 낚시터로, 조용한 휴식처로, 연인들의 데이트 코스로도 유명하다.</p>
</div>
</div>
<div class="cont_more"><button class="btn_more" title="내용더보기" type="button">내용 <span>더보기</span></button></div>
</div>
<button class="btn_modify" onclick="goJikimi();"><span>관광정보 수정요청</span></button>
</div>
<div class="inr">
<p>''엉''이란 제주도 방언으로 언덕을 뜻하는데, 남원 큰엉은 큰 바위가 바다를 집어 삼킬듯이 입을 크게 벌리고 있는 언덕이라 하여 붙여진 명칭으로 절벽 위는 평지로 부드러운 잔디가 깔려 있다. 바다를 향하여 높이 30m, 길이 200m의 기암절벽이 오랜 세월동안 쉬지 않고 밀려오는 파도를 감싸안은 2개의 자연동굴을 보노라면 세속을 떠난듯한 다른 세계, 때 묻지 않은 자연에 대한 감동을 느낄 수 있다. 해안절벽을 따라

In [21]:
# Step 9. 현재 페이지의 내용을 저장 목록을 만든 후 목록에 있는 내용을 파일에 저장하기

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 내용 컬럼

no = 1

click_cnt = 1    # 현재 페이지 번호

for x in range(1,page_cnt+1) :
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)
    
    for i in range(1,11):  
        
        if no > cnt :
            break
            
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%s]/div[2]/div/a""" % i).click( )
        time.sleep(2)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        content_list = soup.find('div','wrap_contView')
        content_top = content_list.find('div','area_txtView top ms_detail').find('div','inr_wrap').find('div','inr')
#        content_top = content_list.find('div','inr_wrap').find('div','inr')
        con_1 = content_top.find('p').get_text()
        print(no , ':  ', con_1)
        
        print("\n")
        no2.append(no)
        contents.append(con_1)
    
        driver.back( )
        time.sleep(2)
        
       
        
        no += 1
        
    if click_cnt > page_cnt+1 :
            break
        
    click_cnt += 1
    
    if (click_cnt % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
    else :
            driver.find_element_by_link_text("""%s""" %click_cnt).click() # 다음 페이지번호 클릭
    time.sleep(2)

1 페이지 내용 수집 시작합니다 =======================
1 :   ''엉''이란 제주도 방언으로 언덕을 뜻하는데, 남원 큰엉은 큰 바위가 바다를 집어 삼킬듯이 입을 크게 벌리고 있는 언덕이라 하여 붙여진 명칭으로 절벽 위는 평지로 부드러운 잔디가 깔려 있다. 바다를 향하여 높이 30m, 길이 200m의 기암절벽이 오랜 세월동안 쉬지 않고 밀려오는 파도를 감싸안은 2개의 자연동굴을 보노라면 세속을 떠난듯한 다른 세계, 때 묻지 않은 자연에 대한 감동을 느낄 수 있다. 해안절벽을 따라 2km의 산책길, 전망대, 간이휴게실, 기초체력단련시설, 화장실 등이 마련되어 있다. 갯바위 낚시터로, 조용한 휴식처로, 연인들의 데이트 코스로도 유명하다.


2 :   제주특별자치도 서귀포시에 있는 한식당이다. 대표메뉴는 칼국수이다. 제주도 대표 음식인 보말(고둥/조개류)이 들어가 있다.


3 :   귤림서원은 제사기능을 가진 충암묘(沖菴廟)와 교육기능을 가진 장수당(藏修堂)이 복합되어 이루어진 서원이다. 충암묘는 1578년(선조 11)에 조인후(趙仁後)판관이 충암 김정(沖菴 金淨)을 봉향하기 위해 세운 사묘(祠廟)이다. 충암묘란 사호(祠號)는 김정의 호를 취한 것이며, 당초의 위치는 가락천 동쪽이었다. 장수당(藏修堂)은 이회(李禬)목사가 향인 김진용(金晉鎔)의 건의로 세종 때 한성판윤을 지낸 고득종(高得宗)의 옛 집터에 세웠던 강당이다.귤림서원은 1871년(고종 8) 흥선대원군의 서원철폐령으로 훼철, 1875년(고종12)봄에는 이희충(李熙忠)목사가 장수당 옛 터에 경신재(敬信齋)를 세워서 선비의 자제들이 학업을 닦을 수 있도록 하였다. 그 후1892년(고종 29)에 조천출신 유생 김희정 등이 옛터에 오현의 뜻을 기리고자 조두석(俎豆石)을 세우고 제단을 축조하여 배향했던 곳이 바로 오늘날의 오현단이다.


4 :   제주특별자치도 제주시에 있는 한식당이다. 대표메뉴는 한치 물회이다. 전복요리와 갈치조림도 유명하다.


5 :   제주특별자치도 제주시에 위치한

In [22]:
# 출력 결과를 표(데이터 프레임) 형태로 만들기

korea = pd.DataFrame()
korea['번호']=no2
korea['내용']=contents

# txt 형식의 파일로 저장하기
f = open(ff_name, 'a',encoding='UTF-8')     
f.write(str(contents))
f.close( )
        
# csv 형태로 저장하기
korea.to_csv(fc_name,encoding="utf-8-sig")

# 엑셀 형태로 저장하기
korea.to_excel(fx_name)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )



총 소요시간은 2291.8 초 입니다 
파일 저장 완료: txt 파일명 : c:\doit\data\2020-09-29-09-49-44-대한민국구석구석\2020-09-29-09-49-44-대한민국구석구석.txt 
파일 저장 완료: csv 파일명 : c:\doit\data\2020-09-29-09-49-44-대한민국구석구석\2020-09-29-09-49-44-대한민국구석구석.csv 
파일 저장 완료: xls 파일명 : c:\doit\data\2020-09-29-09-49-44-대한민국구석구석\2020-09-29-09-49-44-대한민국구석구석.xls 


In [23]:
## 성공
print('='*80)
